In [1]:
import pandas as pd
import numpy as np
from scipy.integrate import odeint
from Source.Classes import Model, epidemic
import itables.interactive
import ipywidgets as widgets
from IPython.display import display

<IPython.core.display.Javascript object>

In [2]:
# Define Generalized Logistic Growth Model

def GLM(t, r, p, K, C_0 = 1):
    def GLM_ode(C_t, t):
        return r*pow(C_t, p)*(1-C_t/K)
    GLM_int = odeint(GLM_ode, C_0, t)
    return GLM_int[:,0]

# Define default parameters for search

par0 = {"r": 0.8, "p": 1, "K":"0.2*pop"} # assumed total outbreak size is 20% of pop
parlower = {"r": 0, "p": 0, "K":"0.1*pop"} # assumed minimum outbreak size is 10% of pop
parupper = {"r": 10, "p": 1, "K":"0.8*pop"} # assumed max outbreak size is 80% of pop

# Initialise the Model

GLM_Model = Model("GLM", GLM, par0, parupper, parlower)

In [3]:
# Load data

source = "https://raw.githubusercontent.com/tomwhite/covid-19-uk-data/master/data/covid-19-cases-uk.csv"
utla_raw = pd.read_csv(source) # Raw Upper Tier Local Authority (UTLA) Case Counts utla_raw.to_csv("dailycases.csv")
popdata = pd.read_csv("popdata.csv")

In [4]:
# Clean data

def str2num(s):
    
    '''
    float <- str
    
    Accounts for irregular phrases such as '1 to 4' in the counts data
    by taking the mean of all integers occuring in the phrase.
    '''
    
    assert type(s) is str, "Case counts is not a string"
    counts = [int(n) for n in s.split() if s.isdigit()]
    if len(counts):
        return np.mean(counts)
    return np.nan
        
utla_raw.TotalCases = [str2num(s) for s in utla_raw.TotalCases]
utla_raw.dropna(subset=['TotalCases'],inplace=True)
utla_raw.Date = pd.to_datetime(utla_raw.Date)
utla_raw = utla_raw[utla_raw.Date < pd.to_datetime('today').strftime("%m/%d/%Y")] # remove today's data as it seems to be underreported

popdata = popdata[popdata.Geog!="Country"]
popdata = popdata[popdata.Geog!="Region"]

In [5]:
# Load the data into the Epidemic class

UKcases = epidemic(casedata = utla_raw, popdata = popdata)

In [6]:
# Fitting all city models, including parameter bootstrap. May take a few minutes.

UKcases.fit_all(GLM_Model, verbose=True, bootstrap=True)

Total cities to attempt: 414
Attempted fits: 10, successes: 8
Attempted fits: 20, successes: 18
Attempted fits: 30, successes: 21
Attempted fits: 40, successes: 30
Attempted fits: 50, successes: 30
Attempted fits: 60, successes: 38
Attempted fits: 70, successes: 41
Attempted fits: 80, successes: 49
Attempted fits: 90, successes: 54
Attempted fits: 100, successes: 55
Attempted fits: 110, successes: 56
Attempted fits: 120, successes: 58
Attempted fits: 130, successes: 62
Attempted fits: 140, successes: 64
Attempted fits: 150, successes: 68
Attempted fits: 160, successes: 72
Attempted fits: 170, successes: 79
Attempted fits: 180, successes: 80
Attempted fits: 190, successes: 81
Attempted fits: 200, successes: 82
Attempted fits: 210, successes: 83
Attempted fits: 220, successes: 92
Attempted fits: 230, successes: 102
Attempted fits: 240, successes: 112
Attempted fits: 250, successes: 122
Attempted fits: 260, successes: 128
Attempted fits: 270, successes: 130
Attempted fits: 280, successes:

In [7]:
# View table of fitted parameters (and standard errors)

df = UKcases.popdata.sort_values(by=['r', 'p'],ascending=False)
df.to_csv("paramfit_cache.csv")

df

Code                         Name               Geog    Area  \
137  E06000020           Telford and Wrekin  Unitary Authority   290.0   
380  W06000018                   Caerphilly  Unitary Authority   277.0   
226  E09000020       Kensington and Chelsea     London Borough    12.0   
235  E09000003                       Barnet     London Borough    87.0   
373  W06000011                      Swansea  Unitary Authority   380.0   
..         ...                          ...                ...     ...   
371  W06000009                Pembrokeshire  Unitary Authority  1619.0   
172  E06000031                 Peterborough  Unitary Authority   343.0   
66   E06000013           North Lincolnshire  Unitary Authority   846.0   
256  E06000046                Isle of Wight  Unitary Authority   380.0   
64   E06000010  Kingston upon Hull, City of  Unitary Authority    71.0   

     Population  PopDensity  MedAge          r            p        K  \
137    177799.0       612.0    39.4    2.11164      0.30642   142239   
380    181019.0       653.0    41.6    1.89429     0.121305  18101.9   
226    156197.0     12884.0    39.4    1.68854     0.347414   124958   
235    392140.0      4520.0    37.0    1.54451      0.35433    39214   
373    246466.0       649.0    39.6     1.5424     0.351679  24646.6   
..          ...         ...     ...        ...          ...      ...   
371    125055.0        77.0    48.0   0.132886  9.30611e-11  12505.5   
172    201041.0       585.0    36.5   0.131098            1   160833   
66     172005.0       203.0    44.1   0.130766            1   137604   
256    141538.0       373.0    50.1  0.0978059            1   113230   
64     260645.0      3648.0    35.6  0.0807752            1   208516   

          r_se       p_se     K_se  LatestTotalCases  
137    1.01473   0.209755    63462              40.0  
380   0.851043   0.241034  47380.4              23.0  
226   0.557368  0.0811889    44629             170.0  
235   0.446951   0.068896   139062             124.0  
373   0.905777   0.283197    86263              19.0  
..         ...        ...      ...               ...  
371  0.0780365   0.414301  37546.5               2.0  
172  0.0365707   0.113226  68934.5              24.0  
66   0.0910078   0.295068  55175.8              15.0  
256  0.0760471   0.341268  46283.8              11.0  
64   0.0776732   0.431404  87323.2              11.0  

[156 rows x 14 columns]

In [8]:
# Plot with confidence intervals

fig = UKcases.popdata.iplot(x="r",y="p",mode="markers",
                            size=np.log(UKcases.popdata.LatestTotalCases)*3,
                            xTitle="Growth rate (r)", yTitle="Growth rate dampening (p)",
                            title="Growth rate vs growth rate dampening per city in England + Wales",
                            asFigure=True)

fig.update_yaxes(autorange="reversed")

r_se_up = np.where(UKcases.popdata.r.values+UKcases.popdata.r_se.values>10, 10-UKcases.popdata.r.values, UKcases.popdata.r_se.values)
r_se_down = np.where(UKcases.popdata.r.values-UKcases.popdata.r_se.values<0, UKcases.popdata.r.values, UKcases.popdata.r_se.values)

p_se_up = np.where(UKcases.popdata.p.values+UKcases.popdata.p_se.values>1, 1-UKcases.popdata.p.values, UKcases.popdata.p_se.values)
p_se_down = np.where(UKcases.popdata.p.values-UKcases.popdata.p_se.values<0, UKcases.popdata.p.values, UKcases.popdata.p_se.values)

fig.update_traces(hovertemplate = '<br><b>%{text}</b>'
                                  '<br>Growth rate (r): %{x:.2f}'
                                  '<br>Growth dampening (p): %{y:.2f}',
                  text = [x+" (latest total cases: "+str(int(y))+")" for (x,y) in zip(UKcases.popdata.Name.values, UKcases.popdata.LatestTotalCases.values)],
                  error_y=dict(
                            type='data', # value of error bar given in data coordinates
                            symmetric=False,
                            array = p_se_up,
                            arrayminus = p_se_down,
                            color='rgba(147,112,219,0.1)',
                            visible=True),
                  error_x=dict(
                            type='data', # value of error bar given in data coordinates
                            symmetric=False,
                            array=r_se_up,
                            arrayminus=r_se_down,
                            color='rgba(147,112,219,0.1)',
                            visible=True),
                                )

fig.show()

In [9]:
# Visualise and explore case counts and projections per city, where sufficient data available

city_dropdown = widgets.Dropdown(options = np.sort(UKcases.popdata.Name.values), description="Choose target city: ")
intervals_dropdown = widgets.Dropdown(options = [False, True], description="Include error bars: ")

graph_output = widgets.Output()

def and_filter(city, intervals):
    graph_output.clear_output()
    with graph_output:
        UKcases.fit_target(city, GLM_Model, visualise=True, assess=False, intervals=intervals)

def city_dropdown_eventhandler(change):
    and_filter(change.new, intervals_dropdown.value)

def intervals_dropdown_eventhandler(change):
    and_filter(city_dropdown.value, change.new)
    
city_dropdown.observe(city_dropdown_eventhandler, names='value')
intervals_dropdown.observe(intervals_dropdown_eventhandler, names='value')

display(city_dropdown)
display(intervals_dropdown)

display(graph_output)

Dropdown(description='Choose target city: ', options=('Barking and Dagenham', 'Barnet', 'Barnsley', 'Bath and …

Dropdown(description='Include error bars: ', options=(False, True), value=False)

Output()